In [ ]:
import Pkg # Importa el administrador de paquetes Pkg
Pkg.add("JuMP") # Instala el paquete JuMP

In [ ]:
Pkg.add("GLPK") # Instala el paquete GLPK

In [ ]:
Pkg.build("GLPK")

In [9]:
using JuMP
using GLPK

# CONSTRUCCIÓN DEL MODELO
modelo = Model(with_optimizer(GLPK.Optimizer))

c = [] # beneficios de c/u de los bloques
tons = [] # Tonelaje de cada bloque

# Lectura archivo de bloques



open("kd.blocks") do f
    line =  1
    while !eof(f)
        a = readline(f)        
        list = split(a)
        #if list[5] == "OXOR"
        #    beneficio = (19.29 * parse(Float64, list[10]) * parse(Float64, list[9]) - 8195) * parse(Float64, list[7])
        #else
        #    beneficio = (19.29 * parse(Float64, list[10]) * parse(Float64, list[9]) - 16862) * parse(Float64, list[7])
        #end  
        beneficio = parse(Float64, list[6])
        append!(c, beneficio)
        append!(tons, parse(Float64, list[5]))           
        line += 1
    end
end

# VARIABLES DE DECISIÓN
@variable(modelo, x[1:1060], Bin) # Variable por cada bloque

# FUNCIÓN OBJETIVO
@objective(modelo, Max, sum(c[i]*x[i] for i in 1:1060))

# RESTRICCIONES

# Presedencia

open("newman1.prec") do f
    while !eof(f)
        a = readline(f)        
        list = split(a)
        n = size(list, 1)
        if n != 2 
            @constraint(modelo, parse(Int64, list[2]) * x[parse(Int64, list[1]) + 1] <= sum(x[parse(Int64, list[j]) + 1] for j in 3:(parse(Int64, list[2]) + 2)))
        end        
    end
end

# Capacidades

# Capacidad maxima tonelaje
@constraint(modelo, sum(x[j]*tons[j] for j in 1:1060) <= 2000000)
# Capacidad minima tonelaje
@constraint(modelo, sum(x[j]*tons[j] for j in 1:1060) >= 1400000)

# Resolver modelo
optimize!(modelo)
objective_value(modelo)



# Mostrar salida por pantalla
print("Valor objetivo: ", objective_value(modelo), ". \n\n") # Entrega el valor objetivo óptimo
print("Valor de las variables\n")
for i in 1:1060
    print("Valor x", i, ": ", value(x[i]), "\n")
end

Valor objetivo: 7.419254435380001e6. 

Valor de las variables
Valor x1: 0.0
Valor x2: 0.0
Valor x3: 0.0
Valor x4: 0.0
Valor x5: 0.0
Valor x6: 0.0
Valor x7: 0.0
Valor x8: 0.0
Valor x9: 0.0
Valor x10: 0.0
Valor x11: 0.0
Valor x12: 0.0
Valor x13: 0.0
Valor x14: 0.0
Valor x15: 0.0
Valor x16: 0.0
Valor x17: 0.0
Valor x18: 0.0
Valor x19: 0.0
Valor x20: 0.0
Valor x21: 0.0
Valor x22: 0.0
Valor x23: 0.0
Valor x24: 0.0
Valor x25: 0.0
Valor x26: 0.0
Valor x27: 0.0
Valor x28: 0.0
Valor x29: 0.0
Valor x30: 0.0
Valor x31: 0.0
Valor x32: 0.0
Valor x33: 0.0
Valor x34: 0.0
Valor x35: 0.0
Valor x36: 0.0
Valor x37: 0.0
Valor x38: 0.0
Valor x39: 0.0
Valor x40: 0.0
Valor x41: 0.0
Valor x42: 0.0
Valor x43: 0.0
Valor x44: 0.0
Valor x45: 0.0
Valor x46: 0.0
Valor x47: 0.0
Valor x48: 0.0
Valor x49: 0.0
Valor x50: 0.0
Valor x51: 0.0
Valor x52: 0.0
Valor x53: 0.0
Valor x54: 0.0
Valor x55: 0.0
Valor x56: 0.0
Valor x57: 0.0
Valor x58: 0.0
Valor x59: 0.0
Valor x60: 0.0
Valor x61: 0.0
Valor x62: 0.0
Valor x63: 0.0
Va

Valor x516: 0.0
Valor x517: 0.0
Valor x518: 0.0
Valor x519: 0.0
Valor x520: 0.0
Valor x521: 0.0
Valor x522: 0.0
Valor x523: 0.0
Valor x524: 0.0
Valor x525: 0.0
Valor x526: 0.0
Valor x527: 0.0
Valor x528: 0.0
Valor x529: 0.0
Valor x530: 0.0
Valor x531: 0.0
Valor x532: 0.0
Valor x533: 0.0
Valor x534: 0.0
Valor x535: 0.0
Valor x536: 0.0
Valor x537: 0.0
Valor x538: 0.0
Valor x539: 0.0
Valor x540: 0.0
Valor x541: 0.0
Valor x542: 0.0
Valor x543: 0.0
Valor x544: 0.0
Valor x545: 0.0
Valor x546: 0.0
Valor x547: 0.0
Valor x548: 0.0
Valor x549: 0.0
Valor x550: 0.0
Valor x551: 0.0
Valor x552: 0.0
Valor x553: 0.0
Valor x554: 0.0
Valor x555: 0.0
Valor x556: 0.0
Valor x557: 0.0
Valor x558: 0.0
Valor x559: 0.0
Valor x560: 0.0
Valor x561: 0.0
Valor x562: 0.0
Valor x563: 0.0
Valor x564: 0.0
Valor x565: 0.0
Valor x566: 0.0
Valor x567: 0.0
Valor x568: 0.0
Valor x569: 0.0
Valor x570: 0.0
Valor x571: 0.0
Valor x572: 0.0
Valor x573: 0.0
Valor x574: 0.0
Valor x575: 0.0
Valor x576: 0.0
Valor x577: 0.0
Valor x5